In [ ]:
import pandas as pd
import numpy as np

# =========================
# 0) Carregar base tratada
# =========================
# Se você já tem df carregado e tratado, pode comentar a linha abaixo.
df = pd.read_csv("BASE_1_Manga_Produtos_TRATADA.csv")

# =========================
# 1) Definir métricas (produto x variáveis)
# =========================
metrics = [
    "margem_media_percentual",
    "indice_valor_agregado_1a5",
    "indice_concorrencia_1a5",
    "indice_complexidade_produtiva_1a5",
    "indice_capex_1a5",
    "indice_escalabilidade_1a5",
    "indice_exigencia_regulatoria_1a5",
    "indice_compatibilidade_refugo_1a5",
    "indice_compatibilidade_operacao_fazenda_1a5",
]

# Garantir que todas são numéricas
for c in metrics:
    df[c] = pd.to_numeric(df[c], errors="coerce")

# =========================
# 2) Tabela bivariada: stats por produto
# =========================
product_stats = (
    df.groupby("produto")[metrics]
      .agg(["count", "mean", "median", "std"])
)
product_stats.columns = [f"{col}_{stat}" for col, stat in product_stats.columns]

# Ranking simples só pela margem (útil para comparação)
top_by_margin = product_stats.sort_values("margem_media_percentual_mean", ascending=False)

print("TOP 10 produtos por MARGEM (média):")
display(top_by_margin.head(10))

# =========================
# 3) Score final (para decidir melhor produto)
#    - padroniza métricas (Z-score) e aplica pesos
# =========================
mean_by_prod = df.groupby("produto")[metrics].mean()

# Z-score (padronização)
Z = (mean_by_prod - mean_by_prod.mean()) / mean_by_prod.std(ddof=0)

# Pesos (ajuste se quiser)
# Sinal positivo = "quanto maior melhor"
# Sinal negativo = "quanto menor melhor"
w = {
    "margem_media_percentual": 0.30,                 # maior melhor
    "indice_valor_agregado_1a5": 0.20,               # maior melhor
    "indice_escalabilidade_1a5": 0.20,               # maior melhor
    "indice_compatibilidade_refugo_1a5": 0.05,       # maior melhor
    "indice_compatibilidade_operacao_fazenda_1a5": 0.05, # maior melhor
    "indice_capex_1a5": -0.10,                       # menor melhor
    "indice_complexidade_produtiva_1a5": -0.05,      # menor melhor
    "indice_exigencia_regulatoria_1a5": -0.03,       # menor melhor
    "indice_concorrencia_1a5": -0.02,                # menor melhor
}

score = sum(Z[col] * peso for col, peso in w.items())

ranking = (
    score.sort_values(ascending=False)
         .to_frame("score")
         .join(mean_by_prod.add_suffix("_mean"))
)

print("\nRANKING FINAL (Top 10) - Melhor produto pelo score:")
display(ranking.head(10))

# =========================
# 4) Justificativa do TOP 1 (quais métricas mais puxaram pra cima/baixo)
# =========================
top_prod = ranking.index[0]
contrib = (Z.loc[top_prod, list(w.keys())] * pd.Series(w)).sort_values(ascending=False)

print(f"\nMELHOR PRODUTO (Top 1): {top_prod}")
print("\nContribuição de cada variável no score (maior = ajudou mais):")
display(contrib.to_frame("contribuicao_no_score"))

print("\nMédias desse produto nas métricas:")
display(mean_by_prod.loc[[top_prod]].T.rename(columns={top_prod: "media_do_top1"}))

# =========================
# 5) (Opcional) Ranking por métricas individuais (para leitura rápida)
# =========================
print("\nTOP 5 por cada métrica (média):")
for col in metrics:
    asc = True if col in ["indice_capex_1a5", "indice_complexidade_produtiva_1a5",
                          "indice_exigencia_regulatoria_1a5", "indice_concorrencia_1a5"] else False
    top5 = mean_by_prod[col].sort_values(ascending=asc).head(5)
    print(f"\n{col} (melhor = {'MENOR' if asc else 'MAIOR'}):")
    display(top5.to_frame("media"))


TOP 10 produtos por MARGEM (média):


,margem_media_percentual_count,margem_media_percentual_mean,margem_media_percentual_median,margem_media_percentual_std,indice_valor_agregado_1a5_count,indice_valor_agregado_1a5_mean,indice_valor_agregado_1a5_median,indice_valor_agregado_1a5_std,indice_concorrencia_1a5_count,indice_concorrencia_1a5_mean,indice_concorrencia_1a5_median,indice_concorrencia_1a5_std,indice_complexidade_produtiva_1a5_count,indice_complexidade_produtiva_1a5_mean,indice_complexidade_produtiva_1a5_median,indice_complexidade_produtiva_1a5_std,indice_capex_1a5_count,indice_capex_1a5_mean,indice_capex_1a5_median,indice_capex_1a5_std,indice_escalabilidade_1a5_count,indice_escalabilidade_1a5_mean,indice_escalabilidade_1a5_median,indice_escalabilidade_1a5_std,indice_exigencia_regulatoria_1a5_count,indice_exigencia_regulatoria_1a5_mean,indice_exigencia_regulatoria_1a5_median,indice_exigencia_regulatoria_1a5_std,indice_compatibilidade_refugo_1a5_count,indice_compatibilidade_refugo_1a5_mean,indice_compatibilidade_refugo_1a5_median,indice_compatibilidade_refugo_1a5_std,indice_compatibilidade_operacao_fazenda_1a5_count,indice_compatibilidade_operacao_fazenda_1a5_mean,indice_compatibilidade_operacao_fazenda_1a5_median,indice_compatibilidade_operacao_fazenda_1a5_std
produto,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Aroma / extrato natural,45,32.234651,34.05,10.323201,45,3.000000,3.0,0.959403,45,2.888889,3.0,1.070731,45,3.266667,3.0,1.175508,45,2.955556,3.0,1.347650,45,3.177778,3.0,1.093137,45,2.733333,3.0,1.338249,45,2.022222,1.0,1.215223,45,2.466667,2.0,1.235829
Purê de manga asséptico,59,31.058289,33.21,10.363536,59,2.957627,3.0,0.778416,59,2.983051,3.0,0.973645,59,3.186441,3.0,1.121492,59,2.661017,3.0,1.267773,59,2.932203,3.0,1.080619,59,2.728814,3.0,1.095925,59,2.338983,2.0,1.254099,59,2.644068,2.0,1.348853
Concentrado de manga,63,30.600005,32.27,10.613672,63,2.992063,3.0,0.785412,63,3.190476,3.0,1.045067,63,3.095238,3.0,1.027278,63,2.682540,3.0,1.147584,63,3.031746,3.0,1.106699,63,2.888889,3.0,1.079294,63,2.238095,2.0,1.227564,63,2.603175,2.0,1.350504
Cubos IQF,51,30.385269,32.17,10.488711,51,2.950980,3.0,0.743336,51,3.000000,3.0,1.077033,51,3.019608,3.0,1.067524,51,2.529412,2.0,1.270479,51,2.901961,3.0,1.187517,51,2.882353,3.0,1.336369,51,2.137255,2.0,1.149254,51,2.745098,2.0,1.521094
Óleo / manteiga de semente de manga,51,30.314748,30.73,11.106946,51,3.019608,3.0,0.793478,51,3.098039,3.0,1.153341,51,3.039216,3.0,1.057559,51,2.960784,3.0,1.232246,51,3.019608,3.0,1.256825,51,3.000000,3.0,1.264911,51,2.294118,2.0,1.237645,51,2.764706,3.0,1.305193
Chutney de manga,46,30.018688,31.36,11.205545,46,3.206522,3.0,0.833840,46,2.978261,3.0,1.144827,46,3.239130,3.0,1.177281,46,2.739130,3.0,1.289871,46,2.956522,3.0,1.134441,46,3.173913,3.0,1.216632,46,2.260870,2.0,1.237107,46,2.413043,2.0,1.375598
Ração / ingrediente para ração animal,45,29.240794,29.54,11.484562,45,2.766667,3.0,0.787689,45,2.866667,3.0,1.254084,45,2.888889,3.0,1.005038,45,2.777778,3.0,1.241130,45,2.711111,3.0,1.160373,45,2.644444,3.0,1.190026,45,2.600000,3.0,1.303840,45,2.644444,3.0,1.190026
Corante natural,52,29.201293,26.66,10.300689,52,2.971154,3.0,0.723645,52,3.269231,3.0,1.254254,52,3.173077,3.0,1.097612,52,2.942308,3.0,1.243232,52,3.057692,3.0,1.109910,52,2.769231,3.0,1.230581,52,2.230769,2.0,1.307825,52,2.711538,3.0,1.288509
Fibra alimentar de manga,65,29.079683,29.05,11.398388,65,2.976923,3.0,0.757415,65,3.184615,3.0,1.184434,65,3.046154,3.0,0.975173,65,2.723077,3.0,1.231205,65,2.800000,3.0,1.092589,65,2.907692,3.0,1.246534,65,2.246154,2.0,1.199359,65,2.430769,2.0,1.060207



RANKING FINAL (Top 10) - Melhor produto pelo score:


,score,margem_media_percentual_mean,indice_valor_agregado_1a5_mean,indice_concorrencia_1a5_mean,indice_complexidade_produtiva_1a5_mean,indice_capex_1a5_mean,indice_escalabilidade_1a5_mean,indice_exigencia_regulatoria_1a5_mean,indice_compatibilidade_refugo_1a5_mean,indice_compatibilidade_operacao_fazenda_1a5_mean
produto,,,,,,,,,,
Aroma / extrato natural,0.861518,32.234651,3.000000,2.888889,3.266667,2.955556,3.177778,2.733333,2.022222,2.466667
Concentrado de manga,0.594451,30.600005,2.992063,3.190476,3.095238,2.682540,3.031746,2.888889,2.238095,2.603175
Purê de manga asséptico,0.583190,31.058289,2.957627,2.983051,3.186441,2.661017,2.932203,2.728814,2.338983,2.644068
Chutney de manga,0.545672,30.018688,3.206522,2.978261,3.239130,2.739130,2.956522,3.173913,2.260870,2.413043
Cubos IQF,0.468844,30.385269,2.950980,3.000000,3.019608,2.529412,2.901961,2.882353,2.137255,2.745098
Óleo / manteiga de semente de manga,0.451343,30.314748,3.019608,3.098039,3.039216,2.960784,3.019608,3.000000,2.294118,2.764706
Molho agridoce / culinário,0.364993,28.631949,3.009091,3.000000,3.200000,2.945455,3.327273,2.818182,1.963636,2.672727
Geleia/compota,0.325642,28.757421,2.866667,3.016667,2.966667,2.750000,3.150000,2.766667,2.250000,2.966667
Biocomposto / fertilizante orgânico,0.287431,28.803366,2.711111,3.222222,2.866667,2.888889,3.333333,2.555556,2.488889,2.666667



MELHOR PRODUTO (Top 1): Aroma / extrato natural

Contribuição de cada variável no score (maior = ajudou mais):


...


indice_compatibilidade_refugo_1a5 (melhor = MAIOR):


,media
produto,
Ração / ingrediente para ração animal,2.600000
Esfoliante natural,2.568966
Sorbet / picolé,2.500000
Biocomposto / fertilizante orgânico,2.488889
Manga desidratada em pedaços,2.482759



indice_compatibilidade_operacao_fazenda_1a5 (melhor = MAIOR):


,media
produto,
Geleia/compota,2.966667
Base cosmética (extrato),2.777778
Óleo / manteiga de semente de manga,2.764706
Bebida funcional com manga,2.760000
Cubos IQF,2.745098


: 